# *The original DCGAN paper recommends some practices for stable training process.*
A variation of those is given below.

In [1]:
from keras.models import Sequential

In [ ]:
'''
   1    Downsample using strided convolutions instead of pooling in the generator.
        A stride of (2,2) has the effect of reducing each dimension of the image by half,
        effectly resultuing in a feature-man of a quarter size.

        for instance, lets say the inputs are colored (3 channel) images, each of size 64 x 64
        a conv2d layer of with a (2,2) stride will output an image of 32 x 32 for each of the filters

        note that the formula for each output dim is : Output dim_i = ((Input dim_i − Filter dim_ )​ / Strides ) + 1

        this means that for a height and width of 64, with the 3,3 kernel size, both height and weight are :

        ((64 - 3) / 2 ) + 1 = 61/2 + 1 = 30.5 + 1 = 31.5

        the padding = "same" parameter will for this to be rounded up (instead of the default of rounding down)
        to give an output with dims (32, 32, no. of filters)
'''

from keras.layers import Conv2D
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(64, (3,3), strides = (2,2), padding = "same", input_shape = (64, 64, 3)))
model.summary()

del(model) # so that i can run this cell multiple times without adding more layers to the same model

In [ ]:
'''
    2   Upsampling using strided transpose convolutions instead of UpSampling layers in the generator.
        A stride of (2,2), which works as output stride for this layer, has the effect of upsampling the
        dimensions - in fact doubling them when used with padding = "same".

        an input of (64,64,3) then produces the output of (128, 128, no. of filters)

        recall how upsampling actually works and note that the filters are used to output a smaller matrix for conv2d
        while the filters are used to output a smaller matrix for the conv2dtranspose
'''

from keras.layers import Conv2DTranspose

model = Sequential()

model.add(Conv2DTranspose(64, (4,4), strides = (2,2), padding = "same", input_shape = (64, 63, 3)))

model.summary()

del(model)

In [ ]:
'''
3   Use leakyReLU instead of relu.
    leakyReLU can be used in place of relu in both the G (generator) and the D (discriminator)

    the negative_slope for this activation can be specified as an optional parameter
    recommended value is 0.2, the default is 0.2

    note that the example below used a conv2d, so its for the discriminator.
    and the leakyReLU activation causes an output of (32, 32, no. of filters)

    leakyReLU, because of the leaky part, accepts some input below the cut-off, unlike relu
    this helps in avoiding getting stuck, helping in the training
'''

from keras.layers import LeakyReLU
from keras.layers import Conv2D

model = Sequential()

model.add(Conv2D(64, (3,3), strides = (2,2), padding = "same", input_shape = (64,64,3)))
model.add(LeakyReLU(0.3))

model.summary()
del(model)

In [ ]:
'''
4   Use batch normalization to standardize the activation outputs from the previous layers
    to have a 0 mean and 1 standard deviation, before passing it to the next layer

    this can be applied to both G and D

    for D, this means sandwiching a batch normalization layer between
    the conv2d downsampler and the leakyrelu activation

    the values of 0 mean and 1 S.D are defaults. check keras documentation for clarity.
'''

from keras.layers import LeakyReLU
from keras.layers import Conv2D
from keras.layers import BatchNormalization

model = Sequential()

model.add(Conv2D(64, (3,3), strides = (2,2), padding = "same", input_shape = (64,64,3)))
model.add(BatchNormalization())
model.add(LeakyReLU(0.3))

model.summary()
del(model)

In [ ]:
'''
5   Use a Random 0-centered Gaussian distribution for initialising the weights
    a random normal bell-shaped distribution of mean 0 and SD = 0.02 is recommended

    this can also be done for both G and D. The example shows for G

    these weight initialisers can be used for each layer

    for the RandomNormal initializer, mean = 0, stddev = 0.05 are defaults.

    the initializer can be specified as an argument in the layers. check keras docs for clarity.
'''

from keras.initializers import RandomNormal
from keras.layers import LeakyReLU
from keras.layers import Conv2DTranspose
from keras.layers import BatchNormalization

rand_gaussian = RandomNormal(mean = 0, stddev = 0.02)

model = Sequential()
model.add(Conv2DTranspose(64, (3,3), strides = (2,2), padding = "same", input_shape = (64,64,3), kernel_initializer = rand_gaussian))
model.add(BatchNormalization())
model.add(LeakyReLU(0.3))

model.summary()

del(model)

In [ ]:
'''
6   Using Adam to optimise the weights by SGD
    with the recommended learning_rate and beta_1 momentum as:
        learning_rate = 0.0002 (defualt = 0.001)
        beta_1 = 0.5 (default = 0.9)

    this is done for both models, G and D

    note that the model can be saved as an h5 after compilation,
    this is useful as it is saved with the specified optimizer
    and can be loaded later

    note that the loss used is binary_crossentropy, that is for a binary classifier,
    and rightly so since the conv2d is used by the discriminiator.
'''

from keras.optimizers import Adam
from keras.layers import Conv2D

model = Sequential([
    Conv2D(64, (3,3), strides = (2,2), padding = "same", input_shape = (64,64,3))
])

opt = Adam(lr = 0.0002, beta_1 = 0.5)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ['accuracy'])

del(model)

In [7]:
'''
7   Use the hyperbolic tangent activation function as the output from the generator model.
    it is also recommended that real images used to train the discriminator
    are scaled so that their pixel values are in the range [-1,1].

    This is so that the discriminator will always receive images as input, real and fake,
    that have pixel values in the same range.

    Typically, image data is loaded as a NumPy array such that pixel values are 8-bit unsigned
    integers in the range [0, 255].
    First, the array must be converted to floating point values, then rescaled to the required range.
'''

from numpy.random import randint

# scale imgs to [-1, 1]
def scale_imgs(imgs):
    imgs = imgs.astype('float32')     # unit8 to float32
    imgs = (imgs - 127.5) / 127.5     # normalize pixel values between -1 and 1
    return imgs

# define one color img of size 28 x 28
imgs = randint(0, 256, 28*28*3) # a 28,28,3 img can be flattened as a 1d vector of size 28*28*3, with values between 0 and 256 (exclusive)
imgs = imgs.reshape((1,28,28,3)) # one img

print(imgs.min(), imgs.max())
scaled = scale_imgs(imgs)

print(scaled.min(), scaled.max())

0 255
-1.0 1.0
